In [25]:
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
import datetime
from PIL import Image

def draw_graph(id,*args, **kwargs):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    id=id,*args,**kwargs
                    )
                ])
            ),
        ])

# Read in files
# Need pandas version > 1.4 and pyarrow installed
RATINGS = pd.read_csv('TV_Ratings_onesheet.csv',engine='pyarrow')
GAMES = pd.read_csv('games_flat_xml_2012-2018.csv',engine='pyarrow')
CAPACITY = pd.read_csv('capacity.csv',engine='pyarrow')

# clean these duration formats
GAMES.loc[679,'duration'] = '3:07'
GAMES.loc[579,'duration'] = '3:14'
GAMES.loc[624,'duration'] = '3:05'
GAMES.loc[773,'duration'] = '2:11'
GAMES.loc[312,'duration'] = '0:00'
GAMES.loc[483,'duration'] = '3:25'
GAMES.loc[491,'duration'] = '0:00'
GAMES.loc[781,'duration'] = '3:00'

# function to convert duration into minutes
def duration_minutes(time):
    hour = np.int16(time.split(':')[0]) * 60
    minutes = np.int16(time.split(':')[1])
    return hour+minutes

# use function on duration column
GAMES['duration_minutes'] = GAMES['duration'].apply(duration_minutes)

# this attendance value is a typo
GAMES.loc[GAMES['attend'] > 200000,'attend'] = 71004

# merge the 3 datasets
MERGED = pd.merge(GAMES,RATINGS,how='inner',on='TeamIDsDate')
# capacity is a custom dataset
MERGED = pd.merge(MERGED,CAPACITY,on=['homename','stadium'])

# a KPI we will use is perecent of capacity
MERGED['Percent_of_Capacity'] = MERGED['attend']/MERGED['Capacity']

# compute total tds scored
MERGED['total_td'] = MERGED[['rush_td_home','pass_td_home','rush_td_vis','pass_td_vis']].sum(axis=1)
# compute total points scored
MERGED['total_pts'] = MERGED[['score_home','score_vis']].sum(axis=1)
# compute point differential
MERGED['pts_diff'] = np.abs(MERGED['score_home'] - MERGED['score_vis'])

#Weather: Categorizing
MERGED.loc[MERGED['weather'].str.contains('Sunny|Clear|fair|beautiful|nice', case = False)==True,'weather'] = 'Clear'
MERGED.loc[MERGED['weather'].str.contains('Cloudy|cldy|clouds|foggy|overcast|Haze', case = False)==True,'weather'] = 'Cloudy'
MERGED.loc[MERGED['weather'].str.contains('Rain|Showers|storms|scattered', case = False)==True,'weather'] = 'Rain'
MERGED.loc[MERGED['weather'].str.contains('Roof Closed|indoors|indoor|dome', case = False)==True,'weather'] = 'Indoors'
MERGED.loc[MERGED['weather'].str.contains('Humidity|Humid|Hot|Warm|Muggy', case = False)==True,'weather'] = 'Hot'
MERGED.loc[MERGED['weather'].str.contains('Cool', case = False)==True,'weather'] = 'Cold'
MERGED['weather'] = MERGED['weather'].replace({np.nan:'Unknown'})
MERGED['weather'] = MERGED['weather'].replace({'':'Unknown'})

# Need to be able to make summary statistics by SEC team
# List of SEC teams
SEC_teams = ['Alabama','Arkansas','Auburn','Florida','Mississippi State','Kentucky','South Carolina',
             'Ole Miss','Georgia','Tennessee','Texas A&M','LSU','Vanderbilt','Missouri']

# One hot-encode SEC teams
# A column for every team
# 1 if that team was in the game (home or away) 0 otherwise
for i in SEC_teams:
    MERGED[i] = np.where(MERGED['Matchup_Full_TeamNames'].str.contains(i),1,0)

# get summary statistics for each team
d = []
for i in SEC_teams:
    df = MERGED.loc[MERGED[i]==1,:] 
    d.append({
        'Team':i,
        'AvgViews':df['VIEWERS'].mean(),
        'Avgattend':df['Percent_of_Capacity'].mean()
    })
# turn the list of dictionaries into a dataframe
df = pd.DataFrame(d)
# sort the data frame by team name
df.sort_values('Team',inplace=True)

viewership = px.bar(data_frame=df,x='Team',y='AvgViews')
viewership.add_hline(df['AvgViews'].mean(),
            line_dash='dot',
            annotation_text="Average: "+str("{:,}".format(np.int64(df['AvgViews'].mean()))), 
            annotation_position="top right",
            annotation_font_size=12,
            annotation_font_color="black")
viewership.update_layout(title={
            'text':"Average TV Views Per Game by School",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        title_font_color="black",
        xaxis_title="School",
        yaxis_title="Number of Views")
viewership.layout.template = 'plotly_white'


attendance = px.bar(data_frame=df,x='Team',y='Avgattend')
attendance.add_hline(df['Avgattend'].mean(),
            line_dash='dot',
            annotation_text="Average: "+str(np.round(df['Avgattend'].mean(),2)), 
            annotation_position="top right",
            annotation_font_size=12,
            annotation_font_color="black")
attendance.update_layout(title={
            'text':"Average Percent of Capacity Per Game by School",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        title_font_color="black",
        xaxis_title="School",
        yaxis_title="Percent of Capacity")
attendance.layout.template = 'plotly_white'

In [1]:
import pandas as pd
pd.__version__

'1.4.1'

In [3]:
import pyarrow

ModuleNotFoundError: No module named 'pyarrow'

In [3]:
import plotly.graph_objects as go

In [24]:
team = 'Tennessee'
fig = go.Figure(
        data=go.Bar(x=MERGED[(MERGED['homename'] == team) |( MERGED['visname'] == team)]['date'],
                    y=MERGED[(MERGED['homename'] == team) |( MERGED['visname'] == team)]['Percent_of_Capacity']))
fig_wig = go.FigureWidget(fig)
fig.show()

In [22]:
MERGED[(MERGED['homename'] == 'Tennessee') |( MERGED['visname'] == 'Tennessee')]['Percent_of_Capacity']
#[i for i in MERGED.columns]
#MERGED['homename']

7      1.000000
19     1.000000
26     1.000000
47     0.837104
67     0.938120
99     0.914946
108    0.965904
112    0.861100
113    0.947606
114    0.976302
115    0.945664
116    0.957533
117    1.000000
118    0.985916
119    1.000000
120    1.000000
121    1.000000
122    0.988385
123    0.996886
124    1.000000
125    0.935250
126    1.000000
127    0.948934
128    1.000000
129    0.934420
130    1.000000
131    1.000000
132    1.000000
182    1.034215
197    1.000000
204    1.000000
209    1.000000
217    1.000000
261    1.000000
283    0.923673
294    0.941912
298    0.925542
307    0.885708
314    1.057845
326    1.026078
332    0.865335
341    0.996631
346    0.991644
377    1.072130
Name: Percent_of_Capacity, dtype: float64